In [1]:
!pip install yfinance
!pip install yahooquery
from yahooquery import Screener
import yfinance as yf   
import pandas as pd
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import random 
from tensorflow.keras.models import load_model


In [59]:
clmns = [
    "Open 12-11","Open 11-10","Open 10-9","Open 9-8","Open 8-7","Open 7-6","Open 6-5","Open 5-4","Open 4-3","Open 3-2","Open 2-1",
    "Close 12-11","Close 11-10","Close 10-9","Close 9-8","Close 8-7","Close 7-6","Close 6-5","Close 5-4","Close 4-3","Close 3-2","Close 2-1",
    "High 12-11","High 11-10","High 10-9","High 9-8","High 8-7","High 7-6","High 6-5","High 5-4","High 4-3","High 3-2","High 2-1",
    "Low 12-11","Low 11-10","Low 10-9","Low 9-8","Low 8-7","Low 7-6","Low 6-5","Low 5-4","Low 4-3","Low 3-2","Low 2-1",
    "AdjV 12-11","AdjV 11-10","AdjV 10-9","AdjV 9-8","AdjV 8-7","AdjV 7-6","AdjV 6-5","AdjV 5-4","AdjV 4-3","AdjV 3-2","AdjV 2-1",
    "Volume 12-11","Volume 11-10","Volume 10-9","Volume 9-8","Volume 8-7","Volume 7-6","Volume 6-5","Volume 5-4","Volume 4-3","Volume 3-2","Volume 2-1",
    "suggestion"
]

def read_syms_from_txt():
  with open("syms.txt", "r") as f:
    lines = f.readlines()
    f.close()
  lst = []
  for line in lines:
    line = line.strip()
    lst.append(line)
  symbols = lst
  return symbols
def get_crypto_syms():
   # 'all_cryptocurrencies_au','all_cryptocurrencies_ca','all_cryptocurrencies_eu','all_cryptocurrencies_gb','all_cryptocurrencies_in',
   screens = [
       'all_cryptocurrencies_us', 'all_cryptocurrencies_au', 'all_cryptocurrencies_ca', 'all_cryptocurrencies_eu', 'all_cryptocurrencies_gb', 'all_cryptocurrencies_in', ]
   s = Screener()
   symbols = []
   for i in screens:
      data = s.get_screeners(i, count=250)
      dicts = data[i]['quotes']
      syms = [d['symbol'] for d in dicts]
      for sym in syms:
        symbols.append(sym)
   # print(len(symbols))
   # pieces = 15
   # new_arrays = np.array_split(symbols, pieces)
   return symbols
def spliting(data):
  X = data.drop(["yes","no"], axis=1)
  y = data[["yes","no"]]
  xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size=0.1)
  print(xTrain.shape, end=" ")
  print(yTrain.shape)
  print(xTest.shape, end=" ")
  print(yTest.shape)
  return xTrain, xTest, yTrain, yTest
def scaler(row):
    scaler = MinMaxScaler(feature_range=(-3, 3))
    row = scaler.fit_transform(row)
    return row
def process(data):
    data = data.dropna()
    row = []
    data = np.array(data)
    llst = [0, 1, 2, 3, 4, 5]

    for i in range(12, data.shape[0]):
        grow = []
        srow = []

        for j in llst:

           srow.append([
               data[i-1][j] - data[i-2][j],
               data[i-2][j] - data[i-3][j],
               data[i-3][j] - data[i-4][j],
               data[i-4][j] - data[i-5][j],
               data[i-5][j] - data[i-6][j],
               data[i-6][j] - data[i-7][j],
               data[i-7][j] - data[i-8][j],
               data[i-8][j] - data[i-9][j],
               data[i-9][j] - data[i-10][j],
               data[i-10][j] - data[i-11][j],
               data[i-11][j] - data[i-12][j]
           ])

        for lst in srow:
            mm = np.array(lst)
            mm = np.reshape(mm, (-1, 1))
            grow.append(scaler(mm))

        sugg = "no"
        if data[i][3] > data[i][0]:
            sugg = "yes"

        arr = np.array(grow).flatten()
        arr = np.append(arr, sugg)
        row.append(arr)


    grow = []
    srow = []
    llst = []
    data = []
    arr = []
    mm = []

    return np.array(row)

In [60]:
def get_data(symbols):
    unattached_dfs = []
    for symbol in symbols:
         data = yf.download(symbol, period="max",
                            interval="1wk", progress=False)
         if data.empty:
              pass
         else:
               if data.shape[0] > 12:
                   unattached_dfs.append(
                       process(data))

    data = np.array(unattached_dfs[0])
    for i in unattached_dfs[1:]:
        data = np.append(data, np.array(i), axis=0)
        
    symbol = []

    data = pd.DataFrame(data, columns=clmns)
    sugg = data["suggestion"]
    data.drop("suggestion",axis=1,inplace=True)
    sugg = pd.get_dummies(sugg)
    data = pd.concat([data,sugg],axis=1)
    data = data.astype(float)

    xTrain, xTest, yTrain, yTest = spliting(data)
    print(data.shape)
    data.to_csv(f"{random.randint(1,10000)}.csv")
    #data = []
    return xTrain, xTest, yTrain, yTest,data



#symbols = read_syms_from_txt()
symbols = get_crypto_syms()

#pieces = 15
#new_arrays = np.array_split(symbols, pieces)
#for symbols in new_arrays[7:]:
#    get_data(symbols)
#symbols = ["btc-usd","eth-usd","trx-usd","bnb-usd","xrp-usd"]
xTrain, xTest, yTrain, yTest,data= get_data(symbols)
#data 


1 Failed download:
- OXEN-AUD: No data found for this date range, symbol may be delisted

1 Failed download:
- VAL-AUD: No data found for this date range, symbol may be delisted

1 Failed download:
- ICP-CAD: No data found for this date range, symbol may be delisted

1 Failed download:
- XNO-CAD: No data found for this date range, symbol may be delisted

1 Failed download:
- OXEN-CAD: No data found for this date range, symbol may be delisted

1 Failed download:
- VAL-CAD: No data found for this date range, symbol may be delisted

1 Failed download:
- ICP-EUR: No data found for this date range, symbol may be delisted

1 Failed download:
- XNO-EUR: No data found for this date range, symbol may be delisted

1 Failed download:
- OXEN-EUR: No data found for this date range, symbol may be delisted

1 Failed download:
- VAL-EUR: No data found for this date range, symbol may be delisted

1 Failed download:
- ICP-GBP: No data found for this date range, symbol may be delisted

1 Failed download

In [61]:
model = Sequential()



model.add(Dense(2000, activation='relu', input_shape=(xTrain.shape[1],)))
model.add(Dense(1000, activation='relu'))
model.add(Dense(1000, activation='relu'))
model.add(Dense(1000, activation='relu'))
model.add(Dense(1000, activation='relu'))
model.add(Dense(1000, activation='relu'))



model.add(Dense(2, activation='sigmoid'))
model.compile(optimizer="adam", loss='binary_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_76 (Dense)            (None, 2000)              134000    
                                                                 
 dense_77 (Dense)            (None, 1000)              2001000   
                                                                 
 dense_78 (Dense)            (None, 1000)              1001000   
                                                                 
 dense_79 (Dense)            (None, 1000)              1001000   
                                                                 
 dense_80 (Dense)            (None, 1000)              1001000   
                                                                 
 dense_81 (Dense)            (None, 1000)              1001000   
                                                                 
 dense_82 (Dense)            (None, 2)               

In [62]:
model.fit(xTrain,yTrain,epochs=35,batch_size=4096,validation_data=(xTest,yTest))

Epoch 1/35
54/54 [==============================] - 148s 3s/step - loss: 0.6607 - accuracy: 0.5926 - val_loss: 0.6133 - val_accuracy: 0.6533
Epoch 2/35
54/54 [==============================] - 141s 3s/step - loss: 0.5657 - accuracy: 0.6948 - val_loss: 0.5333 - val_accuracy: 0.7211
Epoch 3/35
54/54 [==============================] - 146s 3s/step - loss: 0.4859 - accuracy: 0.7551 - val_loss: 0.4670 - val_accuracy: 0.7689
Epoch 4/35
54/54 [==============================] - 140s 3s/step - loss: 0.4011 - accuracy: 0.8140 - val_loss: 0.4109 - val_accuracy: 0.8142
Epoch 5/35
54/54 [==============================] - 140s 3s/step - loss: 0.2992 - accuracy: 0.8735 - val_loss: 0.4127 - val_accuracy: 0.8228
Epoch 6/35
54/54 [==============================] - 141s 3s/step - loss: 0.2311 - accuracy: 0.9092 - val_loss: 0.2521 - val_accuracy: 0.9053
Epoch 7/35
54/54 [==============================] - 144s 3s/step - loss: 0.1692 - accuracy: 0.9385 - val_loss: 0.2266 - val_accuracy: 0.9238
Epoch 8/35
54

In [30]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
model = load_model("/content/drive/MyDrive/[0.23090815544128418, 0.9449216723442078].h5")

In [ ]:
model.fit(xTrain,yTrain,epochs=10,batch_size=55555,validation_data=(xTest,yTest))